In [ ]:
#----------------------------
# Model: SwinCFNet
#----------------------------

# Dependency
import time
import numpy as np
import scipy.io as sio
import h5py 

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from timm.models.layers import to_2tuple, trunc_normal_

import torchprofile
from fvcore.nn import FlopCountAnalysis
from fvcore.nn import parameter_count

from colorama import Fore, Style
from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')
print(Fore.GREEN + '- Done' + Style.RESET_ALL)

- Done


In [15]:
# Device
if torch.cuda.is_available():
    # GPU
    device = torch.device('cuda')
else:
    # CPU
    device = torch.device('cpu')
    
print(Fore.BLUE +'❄️ Compute Device:' + Style.RESET_ALL, end=' ')
print(Fore.GREEN + str(device) + Style.RESET_ALL)    

❄️ Compute Device: cuda


In [16]:
# Dataset Class
class CustomDataset(Dataset):
    def __init__(self, path:str):
        self.mat = h5py.File(name=path, mode='r')['HT']            # (DataSamples, C*H*W)
        self.a = np.reshape(self.mat, (len(self.mat), 2, 32, 32))
    
    # Length function
    def __len__(self):
        return len(self.a)                                         # DataSamples
    
    # Single datasample
    def __getitem__(self, idx):
        sample = self.a[idx]                                        # (C, H, W)
        return torch.tensor(sample, dtype=torch.float32)            # (C, H, W)

In [17]:
# Dataset
torch.manual_seed(2025)
train_data = CustomDataset("/scratch/work/Lanjhi/Data/QuaDRiGa/Data1/train.mat")                                 # (TrainDataSamples, C, H, W)
test_data = CustomDataset("/scratch/work/Lanjhi/Data/QuaDRiGa/Data1/test.mat")                                   # (TestDataSamples, C, H, W)
val_data = CustomDataset("/scratch/work/Lanjhi/Data/QuaDRiGa/Data1/val.mat")                                     # (ValDataSamples, C, H, W)

# DataLoader
torch.manual_seed(2025)
train_loader = DataLoader(train_data, batch_size=200, shuffle=False, pin_memory=True, pin_memory_device="cuda")   # (TrainBatches, C, H, W)
test_loader = DataLoader(test_data, batch_size=1000, shuffle=False, pin_memory=True, pin_memory_device="cuda")   # (TestBatches, C, H, W)
val_loader = DataLoader(val_data, batch_size=1000, shuffle=False, pin_memory=True, pin_memory_device="cuda")     # (ValBatches, C, H, W)


print(f'{Fore.BLUE}{Style.BRIGHT}- DataLoader:{Fore.RESET}', end=' ')
print(f'{Fore.GREEN}Created Suceesfully{Fore.RESET}{Style.RESET_ALL}')

- DataLoader: Created Suceesfully


In [18]:
# NMSE Calculation
def NMSE(sparse_pred, sparse_gt):
    """ 
    Input:  (B, C, H, W)
    Output: NMSE
    Args:
        sparse_pred: (B, C, H, W)
        sparse_gt:   (B, C, H, W)
    Returns:
        x (float): NMSE
    """
    # Convert numpy arrays to PyTorch tensors
    if isinstance(sparse_gt, np.ndarray):
        sparse_gt = torch.tensor(sparse_gt, dtype=torch.float32)                              # (B, C, H, W)
    if isinstance(sparse_pred, np.ndarray):
        sparse_pred = torch.tensor(sparse_pred, dtype=torch.float32)                          # (B, C, H, W)
        
    # Moving to GPU    
    sparse_gt = sparse_gt.to(device)                                                          # (B, C, H, W)
    sparse_pred = sparse_pred.to(device)                                                      # (B, C, H, W)
    
    with torch.no_grad():
        # De-centralize
        sparse_gt = sparse_gt - 0.5                                                           # (B, C, H, W)
        sparse_pred = sparse_pred - 0.5                                                       # (B, C, H, W)
        # Calculate the NMSE
        power_gt = sparse_gt[:, 0, :, :] ** 2 + sparse_gt[:, 1, :, :] ** 2                    # (B, H, W)
        difference = sparse_gt - sparse_pred                                                  # (B, C, H, W)
        mse = difference[:, 0, :, :] ** 2 + difference[:, 1, :, :] ** 2                       # (B, H, W)
        nmse = 10 * torch.log10((mse.sum(dim=(1, 2)) / power_gt.sum(dim=(1, 2))).mean())      # NMSE
        
        return float(nmse) 
    
    
# Window Partition    
def WindowPartition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size

    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape                                                                      # (B, H, W, C)          
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)            # (B, H/window_height, window_height, W/window_width, window_width, C)     
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)  # (Windows, window_height, window_width, C)
    return windows


# Window Reverse
def WindowReverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image

    Returns:
        x: (B, H, W, C)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)      # (B, H/window_size, W/window_size, window_size, window_size, C) 
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)                             # (B, H, W, C)
    return x            

In [19]:
# Patch Embedding Layer: (patch partition ---> linear embedding)
class PatchEmbedding(nn.Module):
    """ 
    Input:  (B, C, H, W) --> (200, 2, 32, 32)
    Output: (B, P1, C1)  --> (200, 256, 64)
    """
    def __init__(self, in_channels=2, patch_dim=64, patch_size=2):                                                # patch_dim = C1 = 64
        super().__init__()
        self.in_channels = in_channels
        self.patch_dim = patch_dim
        self.patch_size = patch_size
        self.proj = nn.Linear(patch_size*patch_size*in_channels, patch_dim)
    
    # forward pass    
    def forward(self, x):
        # Patch Partition
        B, C, H, W = x.shape                                                                                      # (B, C, H, W)
        x = x.permute(0, 2, 3, 1).contiguous()                                                                    # (B, H, W, C)
        x = x.view(B, H//self.patch_size, self.patch_size, W//self.patch_size, self.patch_size, C)                # (B, H/patch, patch, W/patch, patch, C)
        x = x.permute(0, 1, 3, 2, 4, 5).contiguous()                                                              # (B, H/patch, W/patch, patch, patch, C)
        x = x.view(B, -1, self.patch_size*self.patch_size*C)                                                      # (B, H/patch*W/patch, patch*patch*C)
        # Linear Embedding
        x = self.proj(x)                                                                                          # (B, H/patch*W/patch, C1)
        
        return x
    
        
# Patch Merging Layer
class PatchMerging(nn.Module):
    """ 
    Input:  (B, P1, C1) --> (200, 256, 64)
    Output: (B, P2, C2) --> (200, 64, 128)
    """
    def __init__(self, input_resolution=(16, 16), in_dim=64, out_dim=128, patch_size=2):  # patch_dim = C2 = 128
        super().__init__()
        self.input_resolution = input_resolution                                          # (16, 16)
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.patch_size = patch_size
        self.reduction = nn.Linear(patch_size*patch_size*in_dim, out_dim)
    
    # forward pass
    def forward(self, x):
        B, P, C1 = x.shape                                                                 # (B, P, C1)
        H, W = self.input_resolution                                                       # (H, W)
        x = x.view(B, H, W, C1)                                                            # (B, P, C1) --> (B, H, W, C1)
        x0 = x[:, 0::self.patch_size, 0::self.patch_size, :]                               # (B, H//patch, W//patch, C1)       
        x1 = x[:, 1::self.patch_size, 0::self.patch_size, :]                               # (B, H//patch, W//patch, C1) 
        x2 = x[:, 0::self.patch_size, 1::self.patch_size, :]                               # (B, H//patch, W//patch, C1) 
        x3 = x[:, 1::self.patch_size, 1::self.patch_size, :]                               # (B, H//patch, W//patch, C1) 
        x = torch.cat([x0, x1, x2, x3], dim=-1)                                            # (B, H//patch, W//patch, 4*C1)
        x = x.view(B, -1, 4*C1)                                                            # (B, H//patch*W//patch, 4*C1)
        x = self.reduction(x)                                                              # (B, H//patch*W//patch, C2)
        
        return x
    
    
# Patch Division Layer
class PatchDivision(nn.Module):
    """ 
    Input:  (B, P2, C2) --> (200, 64, 128)
    Output: (B, P1, C1) --> (200, 256, 64)
    
    and 
    
    Input:  (B, P1, C1)  --> (200, 256, 64)
    Output: (B, 1024, 2) --> (200, 1024, 2)
    """
    def __init__(self, input_resolution, in_dim, out_dim, patch_size=2):
        super().__init__()
        self.input_resolution = input_resolution                                            # (8, 8)
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.patch_size = patch_size
        self.expansion = nn.Linear(in_dim, patch_size*patch_size*out_dim)
    
    # forward pass    
    def forward(self, x):
        B, P, C2 = x.shape                                                                   # (B, P, C2)
        H, W = self.input_resolution                                                         # (H, W)
        x = self.expansion(x)                                                                # (B, P, patch*patch*out_dim)
        x = x.view(B, H, W, self.patch_size, self.patch_size, self.out_dim)                  # (B, H, W, patch, patch, out_dim)
        x = x.permute(0, 1, 3, 2, 4, 5).contiguous()                                         # (B, H, patch, W , patch, out_dim)         
        x = x.view(B, -1, self.out_dim)                                                      # (B, H*patch*W*patch, out_dim)
        
        return x
    
    
# Multilayer Perceptron
class MLP(nn.Module):
    """
    Input:  (B, P1, C1) --> (200, 256, 64)
    Output: (B, P1, C1) --> (200, 256, 64)
    """
    def __init__(self, dim, hidden_dim_ratio):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim*hidden_dim_ratio)
        self.fc2 = nn.Linear(dim*hidden_dim_ratio, dim)
        self.act = nn.GELU()
        
    # forward pass
    def forward(self, x):
        B, P, C2 = x.shape    # (B, P, C2)
        x = self.fc1(x)       # (B, P, Hidden_dim)
        x = self.act(x)       # (B, P, Hidden_dim)
        x = self.fc2(x)       # (B, P, C2)
        
        return x    
    

# Window Attention
class WindowAttention(nn.Module):
    """ 
    Input:  (B, P1, C1) --> (200, 256, 64)
    Output: (B, P1, C1) --> (200, 256, 64)
    """
    def __init__(self, dim, window_size, num_heads):
        super().__init__()
        self.dim = dim                                                                                                                     # C1/C2
        self.window_size = window_size                                                                                                     # (WindowH, WindowW)
        self.num_heads = num_heads                                                                                                         # Heads
        head_dim = dim // num_heads                                                                                                        # Head dim (dk)
        self.scale = head_dim ** -0.5                                                                                                      # 1\sqrt(dk)

        # Define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))      # ((2*WindowH-1)*(2*WindowW-1), nH)

        # Get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))                                                                         # (2, WindowH, WindowW)
        coords_flatten = torch.flatten(coords, 1)                                                                                          # (2, WindowH*WindowW)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]                                                          # (2, WindowH*WindowW, WindowH*WindowW)
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()                                                                    # (WindowH*WindowW, WindowH*WindowW, 2)
        relative_coords[:, :, 0] += self.window_size[0] - 1                                                                                # Shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)                                                                                  # (WindowH*WindowW, WindowH*WindowW)
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=True)
        self.proj = nn.Linear(dim, dim)

        trunc_normal_(self.relative_position_bias_table, std=0.02)
        self.softmax = nn.Softmax(dim=-1)
    
    # forward pass
    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape                                                                                                               # (num_windows*B, N, C)
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4).contiguous()                     # (num_windows*B, N, 3*C) --> (num_windows*B, N, 3, Heads, C/Heads) --> (3, num_window*B, Heads, N, C/Heads)
        q, k, v = qkv[0], qkv[1], qkv[2]                                                                                                 # make torchscript happy (cannot use tensor as tuple): (num_window*B, Heads, N, C/Heads)

        q = q * self.scale                                                                                                               # (num_window*B, Heads, N, C/Heads)
        attn = (q @ k.transpose(-2, -1))                                                                                                 # (num_window*B, Heads, N, N)

        # Adding learnable relative position bias
        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)        # (WindowH*WindowW, WindowH*WindowW, Heads)
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()                                                                                                                           # (Heads, WindowH*WindowW, WindowH*WindowW)
        attn = attn + relative_position_bias.unsqueeze(0)                                                                                                                                                       # (num_window*B, Heads, N, N) + (Heads, WindowH*WindowW, WindowH*WindowW)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)                                                                                   # (num_window*B, Heads, N, N)
            attn = self.softmax(attn)                                                                                                    # (num_window*B, Heads, N, N)
        else:
            attn = self.softmax(attn)                                                                                                    # (num_window*B, Heads, N, N)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)                                                                                 # (num_windows*B, N, C)
        x = self.proj(x)                                                                                                                 # (num_windows*B, N, C)
        
        return x   


# SwinTransformer Block
class SwinTransformerBlock(nn.Module):
    """ 
    Input:  (B, P1, C1) --> (200, 256, 64)
    Output: (B, P1, C1) --> (200, 256, 64)
    """
    def __init__(self, dim, input_resolution, num_heads, window_size=2, shift_size=0, mlp_ratio=4):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # If window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = nn.LayerNorm(dim)
        self.attn = WindowAttention(dim=dim, window_size=to_2tuple(self.window_size), num_heads=num_heads)
        self.drop_path = nn.Identity()
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = MLP(dim=dim, hidden_dim_ratio=self.mlp_ratio)

        if self.shift_size > 0:
            # Calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))                                                                                 # (1, H, W, 1)
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = WindowPartition(img_mask, self.window_size)                                                          # (nW, window_size, window_size, 1)
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)
    
    # forward pass
    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # Cyclic Shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
            # Partition Windows
            x_windows = WindowPartition(shifted_x, self.window_size)                                                             # (nW*B, window_size, window_size, C)
        else:
            shifted_x = x
            # Partition Windows
            x_windows = WindowPartition(shifted_x, self.window_size)                                                             # (nW*B, window_size, window_size, C)

        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)                                                   # (nW*B, window_size*window_size, C)

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)                                                                 # (nW*B, window_size*window_size, C)

        # Merge Windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)

        # Reverse Cyclic Shift
        if self.shift_size > 0:
            shifted_x = WindowReverse(attn_windows, self.window_size, H, W)                                                      # (B, H', W', C)
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            shifted_x = WindowReverse(attn_windows, self.window_size, H, W)                                                      # (B, H', W', C)
            x = shifted_x
        x = x.view(B, H * W, C)
        x = shortcut + self.drop_path(x)

        # FFN
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x
    

# SwinTransformer
class SwinTransformer(nn.Module):
    """ 
    Input:  (B, P1, C1) --> (200, 256, 64)
    Output: (B, P1, C1) --> (200, 256, 64)
    """
    def __init__(self, dim, input_resolution, num_heads, window_size, shift_size, mlp_ratio):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        # WMSA SwinTransformer
        self.swin_wmsa = SwinTransformerBlock(dim=self.dim, 
                                         input_resolution=self.input_resolution,
                                         num_heads=self.num_heads,
                                         window_size=self.window_size,
                                         shift_size=0,
                                         mlp_ratio=self.mlp_ratio)  
        # SWMSA SwinTransformer
        self.swin_swmsa = SwinTransformerBlock(dim=self.dim, 
                                          input_resolution=self.input_resolution,
                                          num_heads=self.num_heads,
                                          window_size=self.window_size,
                                          shift_size=self.shift_size,
                                          mlp_ratio=self.mlp_ratio)     
        
    # forward pass
    def forward(self, x):
        x = self.swin_wmsa(x)
        x = self.swin_swmsa(x)
            
        return x
    
    
# Encoder Block
class Encoder(nn.Module):
    """
    Input:  (B, C, H, W)  --> (200, 2, 32, 32)
    Output: (B, Codeword) --> (200, Codeword)
    """
    def __init__(self, tx_codeword):
        super().__init__()
        self.patch_embedding = PatchEmbedding(in_channels=2, patch_dim=64, patch_size=2)
        self.swintransformer1 = SwinTransformer(dim=64, input_resolution=(16, 16), num_heads=4, window_size=2, shift_size=1, mlp_ratio=4)
        self.patchmerge = PatchMerging(input_resolution=(16, 16), in_dim=64, out_dim=128, patch_size=2)
        self.swintransformer2 = SwinTransformer(dim=128, input_resolution=(8, 8), num_heads=8, window_size=2, shift_size=1, mlp_ratio=4)
        self.swintransformer3 = SwinTransformer(dim=128, input_resolution=(8, 8), num_heads=8, window_size=2, shift_size=1, mlp_ratio=4)
        self.linear = nn.Linear(8*8*128, tx_codeword)
        
    # forward pass   
    def forward(self, x):
        x = self.patch_embedding(x)
        x = self.swintransformer1(x)
        x = self.patchmerge(x)
        x = self.swintransformer2(x)
        x = self.swintransformer3(x)
        x = x.view(x.shape[0], 8*8*128)
        x = self.linear(x)
        
        return x
    
    
# Decoder Block
class Decoder(nn.Module):
    """
    Output: (B, Codewword) --> (200, Codeword)
    Input:  (B, C, H, W)   --> (200, 2, 32, 32)
    """
    def __init__(self, rx_codeword):
        super().__init__()
        self.linear = nn.Linear(rx_codeword, 8*8*128)
        self.swintransformer1 = SwinTransformerBlock(dim=128, input_resolution=(8, 8), num_heads=8, window_size=2, shift_size=1, mlp_ratio=4)
        self.swintransformer2 = SwinTransformerBlock(dim=128, input_resolution=(8, 8), num_heads=8, window_size=2, shift_size=1, mlp_ratio=4)
        self.patchdiv1 = PatchDivision(input_resolution=(8, 8), in_dim=128, out_dim=64, patch_size=2)
        self.swintransformer3 = SwinTransformerBlock(dim=64, input_resolution=(16, 16), num_heads=4, window_size=2, shift_size=1, mlp_ratio=4)
        self.patchdiv2 = PatchDivision(input_resolution=(16, 16), in_dim=64, out_dim=2, patch_size=2)
    
    # forward pass    
    def forward(self, x):
        B, Codeword = x.shape
        x = self.linear(x)
        x = x.view(B, 64, 128)
        x = self.swintransformer1(x)
        x = self.swintransformer2(x)
        x = self.patchdiv1(x)
        x = self.swintransformer3(x)
        x = self.patchdiv2(x)
        x = x.view(B, 32, 32, 2)
        x = x.permute(0, 3, 1, 2).contiguous()
        
        return x    


# SwinCFNet    
class SwinCFNet(nn.Module):
    def __init__(self, tx_codeword):
        super().__init__()
        self.enc = Encoder(tx_codeword=tx_codeword)
        self.dec = Decoder(rx_codeword=tx_codeword)
    
    # forward pass    
    def forward(self, x):
        x = self.enc(x)
        x = self.dec(x)
        
        return x                      

In [20]:
# Initialization
torch.manual_seed(2025)
model = SwinCFNet(tx_codeword=512).to(device)

# Loss
criterion = nn.MSELoss()

# Optimizer
optimizer = Adam(model.parameters(), lr=1e-4)

In [21]:
#-----------------------------
# Model Params and Flop Count
#-----------------------------

# Model Params Count
def ParamCount(model):
    params=sum(p.numel() for p in model.parameters() if p.requires_grad)
    return print(Fore.BLUE + '- ParamsCount:'+ Style.RESET_ALL,
                Fore.GREEN + f'{params/1000000:.2f}M' + Style.RESET_ALL)


# Model Flops Count
def FlopCount(model):
    test=torch.randn(1, 2, 32, 32).to(device)
    flops=torchprofile.profile_macs(model, test)
    return print(Fore.BLUE + '- FlopsCount:'+ Style.RESET_ALL,
                Fore.GREEN + f'{flops/1000000:.2f}M' + Style.RESET_ALL)


# Params and Flops Count
def ParamFlopCount(model):
    test=torch.randn(1, 2, 32, 32).to(device)
    flop_counter=FlopCountAnalysis(model, test)
    flops=flop_counter.total()
    params = parameter_count(model)['']
    time_start=time.time()
    test_hat=model(test)
    time_end=time.time()
    total_time=time_end-time_start
    return {'Params': f'{params/1000000:.2f}M', 'Flops':f'{flops/1000000:.2f}M', 'RunTime': f'{total_time:.3f}s'}


# Model Params Count
def ParamCountLayers(model):
    params=0
    for name, param in model.named_parameters():
        if param.requires_grad or param.requires_grad:
            params+=param.numel()
            print(f"Layer: {name} | Size: {param.size()} | Parameters: {param.numel()}")
    return  params/1000000

In [22]:
#-------------------------------------------
# Calling ParamCount and FlopCount function  
#-------------------------------------------
  
ParamCount(model)
FlopCount(model)

- ParamsCount: 9.80M
- FlopsCount: 127.17M


In [23]:
#---------------------------------
# Calling ParamFlopCount function
#---------------------------------

ParamFlopCount(model)

Unsupported operator aten::mul encountered 11 time(s)
Unsupported operator aten::add encountered 33 time(s)
Unsupported operator aten::softmax encountered 9 time(s)
Unsupported operator aten::gelu encountered 9 time(s)


{'Params': '9.80M', 'Flops': '127.86M', 'RunTime': '0.006s'}

In [24]:
#----------------
# Model Training
#----------------

num_epochs = 1000
train_losses = []
val_losses = []
nmse_score = []

for epoch in tqdm_notebook(range(num_epochs), desc='Model Training', colour="#0099ff", leave=True):
    # Model Training
    model.eval()
    train_loss = 0
    for data in tqdm_notebook(train_loader, 'Mini Batch Training', colour="#0099ff", leave=False):
        x = data.to(device)
        optimizer.zero_grad()
        model_out = model(x)
        loss = criterion(model_out, x)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*x.size(0)     

    if (epoch+1)%1==0:
        # Model Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data in tqdm_notebook(val_loader, 'Validating The Model', colour="#0099ff", leave=False):
                x = data.to(device)
                output = model(x)
                loss = criterion(output, x)
                val_loss += loss.item()*x.size(0)     

        # Model Testing
        model.eval()
        nmse_error = 0
        with torch.no_grad():
            for data in tqdm_notebook(test_loader, 'Calculating NMSE', colour="#0099ff", leave=False):
                x = data.to(device)
                output = model(x)
                nmse = NMSE(output, x) 
                nmse_error+=nmse   
        
        # Training Loss
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)
        
        # Validation Loss
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        
        # NMSE Score
        avg_nmse = nmse_error/len(test_loader)
        nmse_score.append(avg_nmse) 
        
        # Printing training details
        print(Fore.LIGHTBLUE_EX + f'- Epoch: {epoch+1}/{num_epochs}' + Style.RESET_ALL)
        print(Fore.RED + f'- Train Loss: {train_loss:.9f} | Validation Loss: {val_loss:.9f}' + Style.RESET_ALL, end=' ')
        print(Fore.GREEN + f'| NMSE: {avg_nmse:.7f}' + Style.RESET_ALL)

        # Saving checkpoint
        if avg_nmse<=min(nmse_score):
            params = {'epoch': epoch+1,
                      'model': model.state_dict(),
                      'optimizer': optimizer.state_dict()}
            
            if (epoch+1)<=400:
                torch.save(params, f'model400.pth')
            else:
                torch.save(params, f'model1000.pth')     
    
    # Saving training details            
    np.savetxt(f'nmse_scores.csv', np.array(nmse_score), delimiter=",")
    np.savetxt(f'train_losses.csv', np.array(train_losses), delimiter=",")
    np.savetxt(f'val_losses.csv', np.array(val_losses), delimiter=",")

Model Training:   0%|          | 0/1000 [00:00<?, ?it/s]

Mini Batch Training:   0%|          | 0/525 [00:00<?, ?it/s]

Validating The Model:   0%|          | 0/15 [00:00<?, ?it/s]

Calculating NMSE:   0%|          | 0/30 [00:00<?, ?it/s]

- Epoch: 1/1000
- Train Loss: 0.016544112 | Validation Loss: 0.016980152 | NMSE: -2.0020302


...

- Epoch: 999/1000
- Train Loss: 0.000035393 | Validation Loss: 0.000075379 | NMSE: -26.0157378


Mini Batch Training:   0%|          | 0/525 [00:00<?, ?it/s]

Validating The Model:   0%|          | 0/15 [00:00<?, ?it/s]

Calculating NMSE:   0%|          | 0/30 [00:00<?, ?it/s]

- Epoch: 1000/1000
- Train Loss: 0.000034767 | Validation Loss: 0.000079066 | NMSE: -26.1071073


In [13]:
torch.save(params, f'model685.pth')